In [1]:
import numpy as np
import math
import random
import copy
import fuzzy

In [2]:
ALL_FUZZY_FUNCS = {
    "left_sensor": {
        "name": "left_sensor",
        "left_boundary": 0,
        "right_boundary": 100,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "right_sensor": {
        "name": "right_sensor",
        "left_boundary": 0,
        "right_boundary": 100,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "front_sensor": {
        "name": "front_sensor",
        "left_boundary": 0,
        "right_boundary": 20,
        "mf_names": ["close", "midrange", "far"],
        "is_input": True,
    },
    "velocity": {
        "name": "velocity",
        "left_boundary": 0,
        "right_boundary": 10,
        "mf_names": ["low", "middle", "high"],
        "is_input": False,
    },
    "angle": {
        "name": "angle",
        "left_boundary": -45,
        "right_boundary": 45,
        "mf_names": ["left", "forward", "right"],
        "is_input": False,
    }
}

In [3]:
# y: 1 0 | 0 1 1 0 | 0 1 1 0 | 0 1 1 0 | 0 1
# i: 0 1 | 2 3 4 5 | 6 7 8 9 | ...
def get_ys(size):
    ys = [1, 0]
    for i in range(2, size):
        if (i-2)%4 == 0 or (i-2)%4 == 3:
            ys.append(0)
        else:
            ys.append(1)
    return ys

def xy_split(xs, ys, size):
    xs_split = [np.array([xs[0], xs[1]])]
    ys_split = [np.array([ys[0], ys[1]])]
    split_size = size-2
    
    for i in range(0, split_size):
        k = 4*i
        xs_split.append(np.array([xs[2+k], xs[3+k], xs[4+k], xs[5+k]]))
        ys_split.append(np.array([ys[2+k], ys[3+k], ys[4+k], ys[5+k]]))
    xs_split.append(np.array([xs[-2], xs[-1]]))
    ys_split.append(np.array([ys[-2], ys[-1]]))
    
    return (xs_split, ys_split)
    
    

def random_fuzzy(name, func_names, left, right, alpha, is_input = True):
    size = len(func_names)
    number_of_points = 4*(size-1)
    mean_step = (right - left) / number_of_points
    sigma = alpha*mean_step**0.5
    xs = [left]
    
    for i in range(number_of_points-2):
        new_x = xs[-1] + np.random.normal(mean_step, sigma)
        xs.append(new_x)
    xs.append(right)
    ys = get_ys(number_of_points)
    
    xs_split, ys_split = xy_split(xs, ys, size)
    if is_input:
        return fuzzy.FuzzyInput(name, np.array([fuzzy.MFInput(func_names[i], np.array(xs_split[i]), np.array(ys_split[i])) for i in range(size)])) 
    else:
        return fuzzy.FuzzyOutput(name, np.array([fuzzy.MFOutput(func_names[i], np.array(xs_split[i]), np.array(ys_split[i])) for i in range(size)])) 


In [4]:
def random_fuzzySystem(alpha):
    fuzzy_inputs = {}
    fuzzy_outputs = {}
    
    for fuzzy_key in ALL_FUZZY_FUNCS:
        name = ALL_FUZZY_FUNCS[fuzzy_key]["name"]
        left = ALL_FUZZY_FUNCS[fuzzy_key]["left_boundary"]
        right = ALL_FUZZY_FUNCS[fuzzy_key]["right_boundary"]
        is_input = ALL_FUZZY_FUNCS[fuzzy_key]["is_input"]
        names = ALL_FUZZY_FUNCS[fuzzy_key]["mf_names"]
        
        list_of_functions = []
        if is_input:
            fuzzy_inputs[name] = random_fuzzy(name, names, left, right, alpha, is_input)
        else:
            fuzzy_outputs[name] = random_fuzzy(name, names, left, right, alpha, is_input)
          
    left_sensor = fuzzy_inputs["left_sensor"]
    front_sensor = fuzzy_inputs["front_sensor"]
    right_sensor = fuzzy_inputs["right_sensor"]
    angle = fuzzy_outputs["angle"]
    velocity = fuzzy_outputs["velocity"]
    
    angle_rules = fuzzy.FuzzyRules(np.array([
        fuzzy.Rule(np.array([left_sensor[0], front_sensor[0]]), angle[2]),
        fuzzy.Rule(np.array([right_sensor[0], front_sensor[0]]), angle[0]),
        fuzzy.Rule(np.array([front_sensor[0], left_sensor[3]]), angle[0]),
        fuzzy.Rule(np.array([front_sensor[0], right_sensor[3]]), angle[2])
    ]))

    velocity_rules = fuzzy.FuzzyRules(np.array([
        fuzzy.Rule(np.array([front_sensor[2]]), velocity[2]),
        fuzzy.Rule(np.array([front_sensor[1]]), velocity[1]),
        fuzzy.Rule(np.array([front_sensor[0]]), velocity[0])
    ]))
    
    FSAngle = fuzzy.FuzzySystem(np.array(list(fuzzy_inputs.values())), fuzzy_outputs["angle"], angle_rules)
    FSVelocity = fuzzy.FuzzySystem(np.array(list(fuzzy_inputs.values())), fuzzy_outputs["velocity"], velocity_rules) 
    return (FSAngle, FSVelocity)

In [5]:
name = "angle"
print(random_fuzzy(name, ALL_FUZZY_FUNCS[name]["mf_names"], 0, 100, 1, ALL_FUZZY_FUNCS[name]["is_input"]))

angle : ['[(0.0,1.0), (11.588402456867788,0.0)]', '[(32.75504578364462,0.0), (48.534007921021285,1.0), (66.63987676286621,1.0), (86.48998616554849,0.0)]', '[(94.34678137107979,0.0), (100.0,1.0)]']


In [6]:
a, b = random_fuzzySystem(1)
print(a)
print(b)